# **IMPORTING LIBRARIES**

In [108]:
import requests
import pandas as pd

url = 'https://file.notion.so/f/f/ca71608c-1cc3-4167-857a-24da97c78717/98b0fa4b-2a85-40e0-af20-5a2785c55c92/Untitled.json?id=b6059ad3-5ee5-4f32-84b6-b03e68bdca98&table=block&spaceId=ca71608c-1cc3-4167-857a-24da97c78717&expirationTimestamp=1717668000000&signature=Gknk8PuhKfAfnkTyHLQ6X6ZnJW67ok2swzIeFdW7kO8&downloadName=nested_data.json'  # Replace with your URL

response = requests.get(url)

if response.status_code == 200:
    json_data = response.json()
    print("Sucess")
else:
    print(f"Failed to fetch data. HTTP Status code: {response.status_code}")


Sucess


# **CHECK JSON DATA**

In [109]:
json_data['programs'][0]

{'season': '1842-43',
 'orchestra': 'New York Philharmonic',
 'concerts': [{'Date': '1842-12-07T05:00:00Z',
   'eventType': 'Subscription Season',
   'Venue': 'Apollo Rooms',
   'Location': 'Manhattan, NY',
   'Time': '8:00PM'}],
 'programID': '3853',
 'works': [{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP.67',
   'conductorName': 'Hill, Ureli Corelli',
   'ID': '52446*',
   'soloists': [],
   'composerName': 'Beethoven,  Ludwig  van'},
  {'workTitle': 'OBERON',
   'composerName': 'Weber,  Carl  Maria Von',
   'conductorName': 'Timm, Henry C.',
   'ID': '8834*4',
   'soloists': [{'soloistName': 'Otto, Antoinette',
     'soloistRoles': 'S',
     'soloistInstrument': 'Soprano'}],
   'movement': '"Ozean, du Ungeheuer" (Ocean, thou mighty monster), Reiza (Scene and Aria), Act II'},
  {'workTitle': 'QUINTET, PIANO, D MINOR, OP. 74',
   'ID': '3642*',
   'soloists': [{'soloistName': 'Scharfenberg, William',
     'soloistRoles': 'A',
     'soloistInstrument': 'Piano'},
    {'soloistName': 'Hi

# **TABLES**

**Table Schema**

---Concert              
  1. ConcertID (Primary Key)
  2. Date
  3. EventType
  4. Venue
  5. Location
  6. Time



---Program
  1. SeasonID (Primary Key)
  2. ProgramID (Foreign Key)



---work
  1. WorkID (Primary Key)
  2. WorkTitle
  3. ComposeName
  4. Movment
  5. ProgramID (Foreign Key)



---Season
  1. seasonID  (Primary Key)
  2. SeasonName







In [110]:
json_data['programs'][1].keys()

dict_keys(['season', 'orchestra', 'concerts', 'programID', 'works', 'id'])

In [111]:
df = pd.DataFrame(json_data['programs'])

In [112]:
df.head(3)

,season,orchestra,concerts,programID,works,id
0,1842-43,New York Philharmonic,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",3853,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP....",38e072a7-8fc9-4f9a-8eac-3957905c0002
1,1842-43,New York Philharmonic,"[{'Date': '1843-02-18T05:00:00Z', 'eventType':...",5178,[{'workTitle': 'SYMPHONY NO. 3 IN E FLAT MAJOR...,c7b2b95c-5e0b-431c-a340-5b37fc860b34
2,1842-43,Musicians from the New York Philharmonic,"[{'Date': '1843-04-07T05:00:00Z', 'eventType':...",10785,"[{'workTitle': 'EGMONT, OP.84', 'composerName'...",894e1a52-1ae5-4fa7-aec0-b99997555a37


In [113]:
program = df[['programID','season']]
program.head()

,programID,season
0,3853,1842-43
1,5178,1842-43
2,10785,1842-43
3,5887,1842-43
4,305,1843-44


In [114]:
def extract_works_data(works):
  work_titles = []
  composer_names = []
  conductor_names = []
  IDs = []
  soloists_list = []
  movements = []

  for work in df['works']:
    for items in work:
      work_titles.append(items.get('workTitle', ''))
      composer_names.append(items.get('composerName', ''))
      conductor_names.append(items.get('conductorName', ''))
      IDs.append(items.get('ID', '').replace('*', ''))
      soloists_list.append(items.get('soloists', []))
      movements.append(items.get('movement', ''))

  return work_titles, composer_names, conductor_names, IDs, soloists_list, movements


def extract_concerts_data(concerts):
  concert_dates = []
  event_types = []
  venues = []
  locations = []
  times = []

  for concert in df['concerts']:
    for item in concert:
      concert_dates.append(item.get('Date', ''))
      event_types.append(item.get('eventType', ''))
      venues.append(item.get('Venue', ''))
      locations.append(item.get('Location', ''))
      times.append(item.get('Time', ''))

  return concert_dates, event_types, venues, locations, times

# Extracting works data
work_titles, composer_names, conductor_names, IDs, soloists_list, movements = extract_works_data( df['works'])

# Extracting concerts data
concert_dates, event_types, venues, locations, times = extract_concerts_data(df['concerts'])



In [115]:
works_df = pd.DataFrame({
    'WorkTitle': work_titles,
    'ComposerName': composer_names,
    'ConductorName': conductor_names,
    'ID': IDs,
    'Soloists': soloists_list,
    'Movement': movements
})

concerts_df = pd.DataFrame({
    'Date': concert_dates,
    'EventType': event_types,
    'Venue': venues,
    'Location': locations,
    'Time': times
})




In [116]:
works_df.head()

,WorkTitle,ComposerName,ConductorName,ID,Soloists,Movement
0,"SYMPHONY NO. 5 IN C MINOR, OP.67","Beethoven, Ludwig van","Hill, Ureli Corelli",52446,[],
1,OBERON,"Weber, Carl Maria Von","Timm, Henry C.",88344,"[{'soloistName': 'Otto, Antoinette', 'soloistR...","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons..."
2,"QUINTET, PIANO, D MINOR, OP. 74","Hummel, Johann",,3642,"[{'soloistName': 'Scharfenberg, William', 'sol...",
3,,,,0,[],
4,OBERON,"Weber, Carl Maria Von","Etienne, Denis G.",88343,[],Overture


In [117]:
concerts_df['ConcertID'] = df['programID']

In [118]:
concerts_df.head()

,Date,EventType,Venue,Location,Time,ConcertID
0,1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,3853
1,1843-02-18T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,5178
2,1843-04-07T05:00:00Z,Special,Apollo Rooms,"Manhattan, NY",8:00PM,10785
3,1843-04-22T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,5887
4,1843-11-18T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",None,305
